In [1]:
# transpose : dict[str:str]
transpose = {
    '<':'>',
    '>':'<',
    'e':'et',
    's':'st',
    'et':'e',
    'st':'s',
    'd':'dt',
    'm':'mt',
    'dt':'d',
    'mt':'m',
    'o':'ot',
    'ot':'o',
    '=':'='                 
    }

# symetrie : dict[str:str]
symetrie = {
    '<':'>',
    '>':'<',
    'e':'s',
    's':'e',
    'et':'st',
    'st':'et',
    'd':'d',
    'm':'mt',
    'dt':'dt',
    'mt':'m',
    'o':'ot',
    'ot':'o',
    '=':'='
    }            

# compositionBase : dict[tuple[str,str]:set[str]]             
compositionBase = {        
        ('<','<'):{'<'},
        ('<','m'):{'<'},
        ('<','o'):{'<'},
        ('<','et'):{'<'},
        ('<','s'):{'<'},
        ('<','d'):{'<','m','o','s','d'},
        ('<','dt'):{'<'},
        ('<','e'):{'<','m','o','s','d'},
        ('<','st'):{'<'},
        ('<','ot'):{'<','m','o','s','d'},
        ('<','mt'):{'<','m','o','s','d'},
        ('<','>'):{'<','>','m','mt','o','ot','e','et','s','st','d','dt','='},
        ('m','m'):{'<'},
        ('m','o'):{'<'},
        ('m','et'):{'<'},
        ('m','s'):{'m'},
        ('m','d'):{'o','s','d'},
        ('m','dt'):{'<'},
        ('m','e'):{'o','s','d'},
        ('m','st'):{'m'},
        ('m','ot'):{'o','s','d'},
        ('m','mt'):{'e','et','='},
        ('o','o'):{'<','m','o'},
        ('o','et'):{'<','m','o'},
        ('o','s'):{'o'},
        ('o','d'):{'o','s','d'},
        ('o','dt'):{'<','m','o','et','dt'},
        ('o','e'):{'o','s','d'},
        ('o','st'):{'o','et','dt'},
        ('o','ot'):{'o','ot','e','et','d','dt','st','s','='},
        ('s','et'):{'<','m','o'},
        ('s','s'):{'s'},
        ('s','d'):{'d'},
        ('s','dt'):{'<','m','o','et','dt'},
        ('s','e'):{'d'},
        ('s','st'):{'s','st','='},
        ('et','s'):{'o'},
        ('et','d'):{'o','s','d'},
        ('et','dt'):{'dt'},
        ('et','e'):{'e','et','='},
        ('d','d'):{'d'},
        ('d','dt'):{'<','>','m','mt','o','ot','e','et','s','st','d','dt','='},
        ('dt','d'):{'o','ot','e','et','d','dt','st','s','='}
    }

In [2]:

def transposeSet(S):
    res = set()
    for e in S:
        res.add(transpose[e])
    return res

def symetrieSet(S):
    res = set()
    for e in S:
        res.add(symetrie[e])
    return res

In [3]:
('d','d') in compositionBase

True

In [20]:
def compose(r1,r2):
    if r1 == '=':
        return {r2}
    if r2 == '=':
        return {r1}
    if (r1,r2) in compositionBase:
        return compositionBase[(r1,r2)]
    elif (transpose[r2],transpose[r1]) in compositionBase:
        return transposeSet(compositionBase[(transpose[r2],transpose[r1])])
    elif (symetrie[r1], symetrie[r2]) in compositionBase:
        return symetrieSet(compositionBase[(symetrie[r1], symetrie[r2])])
    else:
        return symetrieSet(transposeSet(compositionBase[(transpose[symetrie[r2]],transpose[symetrie[r1]])]))

In [24]:
compose('ot', 'm')

{'dt', 'et', 'o'}

In [34]:
def compositionSet(S1, S2):
    res = set()
    for e1 in S1:
        for e2 in S2:
            for e in compose(e1, e2):
                res.add(e)
    return res

In [53]:
class Graphe:
    # les indices des noeuds sont 0, 1, 2... n
    def __init__(self,noeuds,relations):
        self.noeuds = noeuds
        self.relations = relations
    def getRelations(self,i,j):
        if (i,j) not in self.relations:
            return transposeSet(self.relations[(j,i)])
        return self.relations[(i,j)]
    def ajouter(self, i, j, R):
        if (j,i) in self.relations:
            self.relation[(j,i)] = transposeSet(R)
        else:
            self.relations[(i,j)] = R

In [58]:
def propagation(G, i, j, R):
    pile = [(i,j)]
    G.ajouter(i,j,R)
    while len(pile) > 0:
        i, j = pile.pop()
        for k in range(len(G.noeuds)):
            if k == i or k == j:
                continue
            Rij = G.getRelations(i,j)
            Rjk = G.getRelations(j,k)
            Rkj = G.getRelations(k,j)
            Rik = G.getRelations(i,k)
            Rki = G.getRelations(k,i)
            nRik = set.intersection(compositionSet(Rij,Rjk), Rik)
            nRkj = set.intersection(compositionSet(Rki,Rij), Rkj)
            if len(nRik) == 0 or len(nRkj) == 0:
                return 'contradiction temporelle'
            if nRik != Rik:
                G.ajouter(i,k,nRik)
                pile.append((i,k))
            if nRkj != Rkj:
                G.ajouter(k,j,nRkj)
                pile.append((k,j))

In [55]:
tuple(enumerate(('A','B','C')))

((0, 'A'), (1, 'B'), (2, 'C'))

In [68]:
G1 = Graphe(tuple(range(3)), {(0,1):{'<'}, (0,2):{'>'}})
G2 = Graphe(tuple(range(3)), {(0,1):{'<'}, (0,2):{'<'}})

In [69]:
propagation(G1, 1, 2, {'='})

'contradiction temporelle'

In [70]:
propagation(G2, 1, 2, {'='})
G2.relations

{(0, 1): {'<'}, (0, 2): {'<'}, (1, 2): {'='}}

In [71]:
tuple(enumerate(('S','L','R')))

((0, 'S'), (1, 'L'), (2, 'R'))

In [82]:
G = Graphe(tuple(range(3)), {(0,2):{},(1,2):{},(1,0):{}})
propagation(G,1,0,{'ot','mt'})
propagation(G,0,2,{'<','>','m','mt'})

'contradiction temporelle'

In [75]:
propagation(G,1,2,{'<','>','o','m','dt','s','st','et','='})

In [77]:
G.relations

{(0, 2): {'<', '>', 'm', 'mt'},
 (1, 0): {'mt', 'ot'},
 (1, 2): {'<', '=', '>', 'dt', 'et', 'm', 'o', 's', 'st'}}